# Data Cleaning

## Overview

Documents the data cleaning pipeline that transforms the raw tick data into cleaned datasets for feature engineering and modeling.

**Data Source:** 100 CSV files containing tick data for 20 liquid U.S. equities across 5 trading days (Oct 2, 3, 6, 7, 8, 2025), collected during regular hours (excluding first/last 5 minutes).

**Goal:** Produce cleaned, standardized datasets without irregular trades, duplicates, and market noise.

| Ticker | Company | Sector | LastClose | ADV60d | ATRpctMed20 | LiquidityPass | PricePass | ATRPass | NearPass |
|:------:|:--------|:--------|----------:|------------------:|--------------:|:--------------:|:----------:|:--------:|:--------:|
| INTC | Intel | Information Technology | 37.01 | 3,304,564,035.88 | 4.83 | True | True | True | False |
| SMCI | Supermicro | Information Technology | 57.69 | 1,630,393,601.76 | 4.13 | True | True | True | False |
| TTD | Trade Desk (The) | Communication Services | 53.30 | 1,048,127,320.14 | 4.38 | True | True | True | False |
| XYZ | Block, Inc. | Financials | 80.17 | 776,195,531.64 | 3.27 | True | True | True | False |
| FCX | Freeport-McMoRan | Materials | 43.15 | 769,067,308.87 | 4.67 | True | True | True | False |
| CCL | Carnival | Consumer Discretionary | 28.70 | 591,964,005.72 | 3.07 | True | True | True | False |
| MCHP | Microchip Technology | Information Technology | 65.64 | 539,277,129.49 | 3.09 | True | True | True | False |
| KVUE | Kenvue | Consumer Staples | 16.53 | 498,341,679.64 | 4.39 | True | True | True | False |
| CNC | Centene Corporation | Health Care | 38.39 | 476,261,026.97 | 4.22 | True | True | True | False |
| DAL | Delta Air Lines | Industrials | 59.70 | 452,152,454.46 | 3.00 | True | True | True | False |
| ON | ON Semiconductor | Information Technology | 50.13 | 444,662,249.68 | 3.47 | True | True | True | False |
| DLTR | Dollar Tree | Consumer Staples | 88.08 | 434,361,819.56 | 3.04 | True | True | True | False |
| PCG | PG&E Corporation | Utilities | 16.53 | 383,304,165.20 | 3.41 | True | True | True | False |
| DXCM | Dexcom | Health Care | 67.80 | 363,548,072.54 | 4.58 | True | True | True | False |
| DOW | Dow Inc. | Materials | 21.94 | 357,634,622.19 | 3.74 | True | True | True | False |
| NCLH | Norwegian Cruise Line Holdings | Consumer Discretionary | 23.58 | 356,042,490.88 | 3.52 | True | True | True | False |
| DECK | Deckers Brands | Consumer Discretionary | 99.00 | 342,460,879.37 | 3.08 | True | True | True | False |
| UBER | Uber | Industrials | 98.05 | 1,629,967,199.43 | 2.93 | True | True | False | True |
| SLB | Schlumberger | Energy | 33.56 | 562,416,049.69 | 2.93 | True | True | False | True |
| EQT | EQT Corporation | Energy | 55.44 | 447,180,508.63 | 2.95 | True | True | False | True |

**Notes:**
- Liquidity threshold (median 60d ADV across S&P 500): \$326,373,156/day  
- ‘NearPass’ = included because it was closest to ATR% band when <20 strict passers exist.  
- For research/education only; data via Yahoo Finance (`yfinance`).

## Pipeline

The cleaning pipeline consists of 7 major stages:

### 1. Load & Validate
- Skip Bloomberg junk headers (first 2 lines)
- Filter out embedded header rows
- Parse timestamps: `America/New_York` $\rightarrow$ `UTC`
- Enforce data types and remove invalid rows (price $\le$ 0, size $\le$ 0)
- Sort by timestamp for temporal consistency

### 2. Deduplicate
- Normalize condition codes (uppercase, sort tokens)
- Remove exact duplicates based on: `(timestamp, type, price, size, exchange, condition_code)`

### 3. Split Streams
- Separate **trades** (`TRADE`) from **quotes** (`BEST_BID`, `BEST_ASK`)
- Process independently for NBBO construction

### 4. Build NBBO
- Construct National Best Bid/Offer timeseries from quotes
- Detect and drop crossed/locked markets (where `nbo < nbb`)
- Forward-fill valid quotes up to **2 seconds** (prevents stale pricing)
- Compute: `mid = 0.5 * (nbb + nbo)` and `spread = nbo - nbb`

### 5. Merge NBBO to Trades
- Use `merge_asof` (backward direction) to attach most recent NBBO to each trade
- Drop trades without valid NBBO within 2-second window
- Compute microstructure flags: `at_bid`, `at_ask`

### 6. Filter by Condition Codes
Apply **priority-based filtering** for composite codes:
- **KEEP:** Blank/`0` (regular trades), `F`/`IS` (ISO sweeps)
- **DROP:** Late reports (`T`), auctions (`R6`), odd-lots (`I`), extended hours (`U`), next-day (`N`), and 20+ other irregular trade types
- **Rule:** If any component in composite code (e.g., `"R6,IS"`) is HARD_EXCLUDE → drop entire trade

### 7. Finalize & Export
- Drop redundant columns (`TradeTime`, original `Spread`)
- Optimize data types: `float32` (prices), `int32` (sizes), `category` (exchanges)
- Export as compressed Parquet files (zstd compression)
- Generate metadata table with quality metrics

## Configuration & Quality Gates

### Key Configuration Parameters
- **Forward-fill cap:** 2 seconds (prevents stale NBBO)
- **Timezone:** America/New_York → UTC conversion
- **Compression:** zstd (efficient for time-series)
- **Parallel processing:** 7 workers (CPU count - 1)
- **Condition codes:** 27 codes mapped to 3 buckets (NEUTRAL, SOFT_INCLUDE, HARD_EXCLUDE)

### Quality Gates

| Metric | Threshold | Purpose |
|--------|-----------|---------|
| **NBBO Coverage** | > 99% | Ensure trades have valid market context |
| **Crossed Quotes** | < 0.5% | Detect market data quality issues |
| **Negative Spreads** | 0 | Enforce bid-ask logic |
| **NaN Values** | 0 | No missing data in final output |
| **Timestamp Order** | Monotonic | Maintain temporal sequence |


> Quality gates produce warnings to allow analysis of data quality issues while continuing processing.


## Test

### Test Results Analysis

The test run processes 2 sample files (CCL for Oct 2-3) to validate the pipeline before full batch processing.

The schema validation confirms proper data types and no missing values in critical fields.


In [ ]:
import os
import sys

os.chdir('..')
print(f"Working directory: {os.getcwd()}")

Working directory: c:\Users\doqui\OneDrive\Documents\cs4641-131-project


In [2]:
!python src/dataCleaning/test_clean_data.py

Testing Data Cleaning Pipeline on Sample Files

Loaded 26 condition codes
Config: ffill_cap=2s

Testing on 2 sample files:
  - data\raw\CCL\CCL_10-2-25.csv
  - data\raw\CCL\CCL_10-3-25.csv

Processing: data\raw\CCL\CCL_10-2-25.csv
2025-10-22 16:09:40,606 [INFO] Processing CCL 10-2-25...
2025-10-22 16:09:54,228 [INFO] [OK] CCL 10-2-25: 13143 final rows (13.62s)

[SUCCESS]
  Raw rows: 201,685
  Final rows: 13,143
  Duplicates removed: 51741 (25.65%)
  NBBO coverage: 100.00%
  Crossed quotes: 0.308%
  Trades dropped by cond: 21430
  Trades dropped (no NBBO): 1
  Processing time: 13.62s

  Output file exists: data\clean\CCL\CCL_10-2-25_clean.parquet
  Loaded 13143 rows from parquet

  Schema:
    ts: datetime64[ns, UTC]
    ticker: object
    type: category
    price: float32
    size: int32
    cond: object
    cond_norm: object
    exch: category
    nbb: float32
    nbo: float32
    nbb_size: int32
    nbo_size: int32
    mid: float32
    spread: float32
    at_bid: int64
    at_ask: in

## Clean

In [3]:
!python src/dataCleaning/clean_data.py

Data Cleaning Pipeline
2025-10-22 16:10:16,376 [INFO] Loading configuration and codebook...
2025-10-22 16:10:16,382 [INFO] Loaded 26 condition codes
2025-10-22 16:10:16,386 [INFO] Found 100 CSV files to process
2025-10-22 16:10:16,386 [INFO] Processing with 7 parallel workers...
2025-10-22 16:15:42,544 [INFO] Wrote metadata to data\clean\metadata.parquet

SUMMARY
Files processed: 100
  Success: 100
  Failed: 0

Total rows:
  Raw: 21,419,919
  Final: 1,130,327

Average statistics:
  Duplicate removal: 33.07%
  NBBO coverage: 99.99%
  Crossed quotes: 0.522%
  Processing time: 22.48s per file

Quality Gates:
  [OK] All files meet NBBO coverage threshold (>99.0%)
  ⚠ WARNING: 45 files with crossed quotes > 0.5%
    - DAL 10-7-25: 0.546%
    - DAL 10-6-25: 0.519%
    - DLTR 10-2-25: 0.520%
    - DAL 10-8-25: 0.606%
    - DLTR 10-6-25: 0.795%
    - DLTR 10-3-25: 0.838%
    - DLTR 10-8-25: 0.779%
    - DLTR 10-7-25: 0.715%
    - DXCM 10-2-25: 0.614%
    - DXCM 10-6-25: 0.750%
    - DXCM 10-8-

## Inspect

In [4]:
!python src/dataCleaning/inspect_cleaned_data.py

CLEANED DATA INSPECTION

Processing Summary:
  Total files: 100
  Successful: 100
  Failed: 0

Data Volume:
  Raw rows: 21,419,919
  Final rows: 1,130,327
  Retention: 5.3%

Quality Metrics (Average):
  NBBO coverage: 99.99%
    - Min: 99.82%
    - Max: 100.00%
  Crossed quotes: 0.522%
    - Min: 0.000%
    - Max: 2.028%
  Duplicate removal: 33.07%
  Trades dropped (cond codes): 2,005,302
  Trades dropped (no NBBO): 140

Performance:
  Avg processing time: 22.48s per file
  Total processing time: 2248.2s

Per-Ticker Summary:
        Total Rows  Avg NBBO Cov %  Avg Crossed %
ticker                                           
INTC        231234           99.99           1.07
SMCI        108111          100.00           1.51
FCX          88471          100.00           0.56
PCG          83556          100.00           0.12
KVUE         75123          100.00           0.20
CCL          65290           99.99           0.20
TTD          54152          100.00           0.95
UBER         51968 